In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("papers.csv")

In [4]:
df.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [5]:
df.shape

(7241, 7)

In [6]:
#reducing the dataset
df = df.iloc[:5000, :]
print(df.shape)

(5000, 7)


### data cleaning

In [7]:
import re
import nltk
#librarty to remove stop words
from nltk.corpus import  stopwords
#library to perform stemming
from nltk.stem.porter import PorterStemmer



In [8]:
EnglishStopWrods = set(stopwords.words("english"))
# creating a list of custom stopword
new_words = ["fig", "figure", "image", "sample", "using", "show", "result", "large", "also", "one", "two", "three", "four", "five", "seven", "eight", "nine"]
EnglishStopWrods = list(EnglishStopWrods.union(new_words))

In [26]:
def cleanText(text):
    #converting the text to lower case
    text = text.lower()
    #remove the html tags using re
    text = re.sub('<.*?>', '', text)
    #remove token other than a-zA-Z
    text = re.sub('[^a-zA-Z]', ' ', text)
    #tokeinzation
    text = nltk.word_tokenize(text)
    #remove the stop words
    text = [word for word in text if not word in EnglishStopWrods]
    #removing the words with length less than 3
    text = [word for word in text if len(word) > 3]
    #performing lemetization
    stemming = PorterStemmer()
    text = [stemming.stem(word) for word in text]

    return " ".join(text)

In [27]:
inputtext  = "This is to hp hop from loving running 23432 @#!@# the Input text<html>"
print(f"The processed text is : \n {cleanText(inputtext)}")


The processed text is : 
 love run input text


In [32]:
df["paper_text_clean"] = df["paper_text"].apply(lambda x: cleanText(x))

### CountVectorizer

For this task, I’ll first use the CountVectorizer method in Scikit-learn to create a vocabulary and generate the word count

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_df=0.95, max_features=5000, ngram_range=(1,3)) # if any toekn appears in more than 95% of the document then ignore it
count_vector = cv.fit_transform(df["paper_text_clean"])

<5000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 3064773 stored elements in Compressed Sparse Row format>

### perfroming tfidf tranformer

converting the text token into vectors

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(smooth_idf=True, use_idf=True) # we want to give meaninful weight to the words with less occurance in document, as this are the keywords
tfidf_output = tfidf.fit_transform(count_vector)